In [1]:
%run stock_dataset_creator.ipynb

In [23]:
def simple_model_trainer(name):
    stock_ds = stock(name=name)

    """Results are not evenly good. ACA.AP is pretty good whereas MSFT is catastrophic

What is left to improve is the choice of the estimator's hyperparamater
such as their quality, the lenght training dataset."""


    """The only indicators taken into account are, for now : 
Relative Strenght Indicator, 
Momentum of Market, 
Bollinger Bands width, 
Rate of Change, 
Prices (not of the forcasted period of course)"""

    X = pd.DataFrame({"RSI" : stock_ds.RSI(), 
                      "MOM" : stock_ds.MOM(), 
                      "Bollinger Width" : stock_ds.Bollinger_width(), 
                      "ROC" : stock_ds.ROC(),
                      "Prices" : stock_ds.price()})

    y = pd.DataFrame({"Prices" : stock_ds.price()})

    X,y = X.dropna(), y.dropna()


    assert len(X)!=0, "X dataset is empty bordel"
    assert len(y)!= 0, "y dataset is empty bordel"

    """"The segmentation of the train/test datasets are arbitrary. 
The forecasting period is expected to be Month+1
We expect to encounter some size issues as markets are not open 365/365 days 
and the closing days are not excatly the same every year"""

    X_train = X["2014-01":"2016-01"]
    y_train = y["2016-02":"2018-02-25"]
    
    X_test_start = "2019-02"
    X_test_end = "2019-09"

    assert X_test_start < X_test_end

    X_test = X[X_test_start:X_test_end]
    y_test = y["2019-03-06":"2019-10"]

    """" Below stands a way of dealing with the size issue of training datasets.
It's a degree 0 way of dealing with it. There are certainly more efficient 
ways to handle it"""

    if len(y_train)-len(X_train)>0 : y_train= y_train[:-abs(len(y_train)-len(X_train))]
    if len(y_train)-len(X_train)<0 : X_train= X_train[:-abs(len(y_train)-len(X_train))]


    from sklearn.svm import SVR
    from sklearn import linear_model
    
    models = {"SVR linear" : SVR(kernel="linear"), 
                "SVR rbf" : SVR(kernel="rbf"), 
                "ridge " :linear_model.Ridge(alpha=.5), 
                "lasso": linear_model.Lasso(alpha=.1)}
    
    
            
    """Alpha paramaters are arbitratry"""
        
    models_fitted = {model : models[model].fit(X_train, np.ravel(y_train)) for model in models.keys()}
    for model in models_fitted.keys() : 
        
        y_pred = models_fitted[model].predict(X_test)
        if len(y_test)-len(y_pred)>0 : 
            y_test= y_test[:-abs(len(y_test)-len(y_pred))]
            if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_test est trop long")
        if len(y_test)-len(y_pred)<0 : 
            y_pred= y_pred[:-abs(len(y_test)-len(y_pred))]
            if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_pred est trop long")
        y_test["pred_"+model] = y_pred

    """"We have to equalize the size of the test datasets"""
    """"
    if len(y_test)-len(y_pred)>0 : 
        y_test= y_test[:-abs(len(y_test)-len(y_pred))]
        if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_test est trop long")
    if len(y_test)-len(y_pred)<0 : 
        y_pred= y_pred[:-abs(len(y_test)-len(y_pred))]
        if len(y_test)-len(y_pred)!=0 : raise ValueError("L'ajustement ne fonctionne pas, y_pred est trop long")
        
    """
            
    return(y_test[X_test_end:])

